# Energy Economic Analysis - ETL

In [1]:
import pandas as pd
import os
import time
import pycountry
import math
import numpy as np
import plotly.graph_objects as go

# Identify Data Files

Identify all spreadsheets that contain data for the scoring model.  Spreadsheets are stored in a folder titled 'Raw Data'.

In [2]:
raw_files = os.listdir('./Raw Data')
print("Total Files: ", len(raw_files))

Total Files:  20


In [3]:
bmi_files = []
for i in raw_files:
    if ('BMI' in i) and ('~' not in i):
        bmi_files.append(i)
print('BMI Files Found: ')
bmi_files

BMI Files Found: 


['BMIComparativeData_MIDDLE EAST.xlsx',
 'BMIComparativeData_ASIA PT 2.xlsx',
 'BMIComparativeData_AFRICA PT 1.xlsx',
 'BMIComparativeData_GLOBAL.xlsx',
 'BMIComparativeData_CARIBBEAN.xlsx',
 'BMIComparativeData_EUROPE PT 1.xlsx',
 'BMIComparativeData_EUROPE PT 2.xlsx',
 'BMIComparativeData_NORTH AMERICA.xlsx',
 'BMIComparativeData_ASIA PT 1.xlsx',
 'BMIComparativeData_AFRICA PT 2.xlsx',
 'BMIComparativeData_LATIN AMERICA.xlsx']

In [4]:
access_files = []
for i in raw_files:
    if ('API' in i) and ('~' not in i):
        access_files.append(i)
print('Access to Electricity Files Found: ')
access_files

Access to Electricity Files Found: 


['API_EG.ELC.ACCS.RU.ZS_DS2_Access to Electricity Rural.xls',
 'API_EG.ELC.ACCS.UR.ZS_DS2_Access to Electricity Urban.xls',
 'API_EG.ELC.ACCS.ZS_DS2_Access to Electricity.xls']

In [5]:
other_files = []
rise_file = ''
survey_file = ''
co2_file = ''
mktpot_file = ''

for i in raw_files:
    if 'RISE' in i:
        rise_file = i
    elif 'Enterprise' in i:
        survey_file = i
    elif 'CO2' in i:
        co2_file = i
    elif 'Potential' in i:
        mktpot_file = i
    elif (i not in bmi_files) and (i not in access_files) and ('~' not in i):
        other_files.append(i)
    else:
        pass

print('Other Files Found:')
print(rise_file)
print(survey_file)
print(co2_file)
print(mktpot_file)
print('\nFiles not included:')
print(other_files)


Other Files Found:
RISE Scores 2017.xlsx
Infrastructure Enterprise Survey.xlsx
CO2 Emissions World Bank 2014.xlsx
Market Potential.xlsx

Files not included:
['.DS_Store', 'Country Region Mapping.xlsx']


# Process BMI Files

In [6]:
# Read in each BMI file 
bmi_df = pd.DataFrame()

drop_cols = ['SeqId', 'Service Name', 'Data Definition', 'Data Source', 'Notes', 'Last Reviewed']

for f in bmi_files:
    print('Processing: ', f)
    path = './Raw Data/' + f
    temp_df = pd.read_excel(path, sheet_name='Yearly Data', header=9)
    temp_df = temp_df.drop(drop_cols, axis=1)
    bmi_df = pd.concat([bmi_df, temp_df], ignore_index=True, sort=False)

bmi_df = bmi_df.drop('Sub-National Entity', axis=1)

Processing:  BMIComparativeData_MIDDLE EAST.xlsx
Processing:  BMIComparativeData_ASIA PT 2.xlsx
Processing:  BMIComparativeData_AFRICA PT 1.xlsx
Processing:  BMIComparativeData_GLOBAL.xlsx
Processing:  BMIComparativeData_CARIBBEAN.xlsx
Processing:  BMIComparativeData_EUROPE PT 1.xlsx
Processing:  BMIComparativeData_EUROPE PT 2.xlsx
Processing:  BMIComparativeData_NORTH AMERICA.xlsx
Processing:  BMIComparativeData_ASIA PT 1.xlsx
Processing:  BMIComparativeData_AFRICA PT 2.xlsx
Processing:  BMIComparativeData_LATIN AMERICA.xlsx


In [7]:
# Display statistics on BMI data
print('BMI DataFrame Shape: ', bmi_df.shape)
print('Unique Geographies: ', bmi_df['Geography'].nunique())
print('Unique Indicators: ', bmi_df['Indicator Name'].nunique())

BMI DataFrame Shape:  (8454, 18)
Unique Geographies:  210
Unique Indicators:  63


In [8]:
# Convert column names to strings
bmi_df.columns = list(map(lambda x: str(x), bmi_df.columns))

In [9]:
bmi_df.head()

,Geography,Indicator Name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,MENA (Region),"ELECTRICITY GENERATION: Generation, Total, TWh",1005.8,1051.5,1112.7,1156.6,1237.6,1311.9,1342,1395.1,1442.6,1495.1,1494.2,1576.1,1648.5,1703.1,1751.7,1798.8
1,MENA (Region),"ELECTRICITY GENERATION: Generation, Total, % y...",8.401,4.537,5.818,3.95,7.002,6.006,2.294,3.959,3.399,3.641,-0.062,5.487,4.588,3.314,2.853,2.689
2,MENA (Region),"ELECTRICITY GENERATION: Generation, Total, KWh...",3049.08,3116.66,3223.16,3274.54,3426.72,3556.53,3566.33,3638.39,3695.68,3765.93,3703.19,3846.56,3964.38,4038.56,4097.77,4152.84
3,MENA (Region),"ELECTRICITY GENERATION: Generation, Thermal, TWh",971.306,1016.47,1074.78,1110.53,1194.26,1267.4,1290.93,1342.93,1387.38,1429.11,1411.56,1474.98,1526.64,1562.81,1603.46,1643.71
4,MENA (Region),"ELECTRICITY GENERATION: Generation, Thermal, %...",8.039,4.65,5.736,3.326,7.54,6.124,1.856,4.028,3.31,3.008,-1.228,4.493,3.502,2.369,2.601,2.51


### Clean up BMI indicator names

Indicator Names are long and descriptive, need to abbreviate them for easier viewing in the final dataframe.

In [10]:
# Read in indicator naming convention from spreadsheet
indicators = pd.read_excel('./Dataset Indicators.xlsx', usecols=['Indicator Name', 'Abbreviated Name'])

In [11]:
# Remove regional data in BMI dataframe
bmi_df = bmi_df[list(map(lambda x: ('(Region)' not in x) and ('Eurozone' not in x), bmi_df['Geography']))]

In [12]:
# Apply abbreviated indicator names
print('Indicator Name replacement progress:')
print('|', '-'*int(math.ceil((len(bmi_df) / 100))), '|')
print('  ', end='')

for i in range(len(bmi_df)):
    bmi_df.iloc[i,1] = list(indicators.loc[bmi_df.iloc[i,1] == indicators['Indicator Name']]['Abbreviated Name'])[0]
    if i % 100 == 0:
        print('*',end='')

Indicator Name replacement progress:
| ---------------------------------------------------------------------------------- |
  **********************************************************************************

In [13]:
# Rename Country Column
bmi_df.rename(columns={'Geography':'Country Name'}, inplace=True)

In [14]:
# Verify indicators abbreviated
bmi_df.head()

,Country Name,Indicator Name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
48,Bahrain,GEN_TOTAL_TWH,21,21.4,21.7,23,24.6,26.1,27.3,28.5,29.7,39.8,41.1,42.4,43.7,45,46.3,47.7
49,Bahrain,GEN_TOTAL_YOY,5.145,2.301,1.3,6.034,6.625,6.293,4.483,4.684,4.186,33.887,3.192,3.143,3.103,2.994,2.974,2.964
50,Bahrain,GEN_TOTAL_KWHPP,16889.5,16774,16707.3,17512.2,18378.3,19025.4,19126.3,19107.1,18951,24264.1,24149.8,24243.2,24495.5,24838.8,25219.4,25586.1
51,Bahrain,GEN_THERMAL_TWH,20.956,21.437,21.718,23.028,24.549,26.094,27.264,28.541,29.736,39.815,41.086,42.377,43.692,45,46.338,47.712
52,Bahrain,GEN_THERMAL_YOY,5.146,2.298,1.307,6.034,6.604,6.294,4.484,4.684,4.186,33.894,3.193,3.143,3.103,2.994,2.974,2.964


### Append country codes

In [15]:
# Identify all countries in BMI data
geo = bmi_df['Country Name'].unique()

In [16]:
# Create dictionary for all countries that don't have correct alpha-3 codes in pycountry
nonfits = {'West Bank and Gaza': 'PSE',
 'North Korea':'PRK',
 'South Korea':'KOR',
 'Cape Verde':'CPV',
 'Central African Rep.':'CAF',
 'Congo (DRC)':'COD',
 'Congo-Brazzaville':'COG',
 'Cote d`Ivoire':'CIV',
 '   Global':'GLOBAL',
 'Trinidad & Tobago':'TTO',
 'Bosnia-Herzegovina':'BIH',
 'Eurozone':'EUROZONE',
 'Laos':'LAO',
 'Macau':'MAC',
'Curacao':'CUW', 
'Sint Maarten':'ANT',
'Niger':'NER',
'Kosovo':'RKS'}

In [17]:
# Create a consolidated dictionary of country alpha3 codes to accelerate iteration step
# Use list(pycountry.countries) to show all countries in library

not_in = []
geo_lookup = {}
for i in geo: 
    try:
        alpha3 = pycountry.countries.search_fuzzy(i)[0].alpha_3
        geo_lookup[i] = alpha3
    except:
        try:
            alpha3 = nonfits[i]
            geo_lookup[i] = alpha3
        except:
            not_in.append(i)

geo_lookup.update(nonfits)

# Report if missing any countries in dictionary
print('Non Matches: ', not_in)

Non Matches:  []


In [18]:
# Create Country Code Column in BMI dataframe
bmi_df['Country Code'] = ''

# Apply country codes using geo_lookup dictionary
print('Alpha-3 country encoding progress:')
print('|', '-'*int(math.ceil((len(bmi_df) / 100))), '|')
print('  ', end='')

geo_col = list(bmi_df.columns).index('Country Code')
for idx, val in enumerate(bmi_df['Country Name']):
    bmi_df.iloc[idx,geo_col] = geo_lookup[val]
    if idx % 100 == 0:
        print('*',end='')

Alpha-3 country encoding progress:
| ---------------------------------------------------------------------------------- |
  **********************************************************************************

In [19]:
# Verify country codes added:
bmi_df.head()

,Country Name,Indicator Name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,Country Code
48,Bahrain,GEN_TOTAL_TWH,21,21.4,21.7,23,24.6,26.1,27.3,28.5,29.7,39.8,41.1,42.4,43.7,45,46.3,47.7,BHR
49,Bahrain,GEN_TOTAL_YOY,5.145,2.301,1.3,6.034,6.625,6.293,4.483,4.684,4.186,33.887,3.192,3.143,3.103,2.994,2.974,2.964,BHR
50,Bahrain,GEN_TOTAL_KWHPP,16889.5,16774,16707.3,17512.2,18378.3,19025.4,19126.3,19107.1,18951,24264.1,24149.8,24243.2,24495.5,24838.8,25219.4,25586.1,BHR
51,Bahrain,GEN_THERMAL_TWH,20.956,21.437,21.718,23.028,24.549,26.094,27.264,28.541,29.736,39.815,41.086,42.377,43.692,45,46.338,47.712,BHR
52,Bahrain,GEN_THERMAL_YOY,5.146,2.298,1.307,6.034,6.604,6.294,4.484,4.684,4.186,33.894,3.193,3.143,3.103,2.994,2.974,2.964,BHR


# Process Electricity Access Files

In [20]:
# Read in Electricity Access Data
access_df = pd.DataFrame()
drop_cols = ['Indicator Code']

for f in access_files:
    print('Processing: ', f)
    path = './Raw Data/' + f
    temp_df = pd.read_excel(path, sheet_name='Data', header=3) 
    temp_df = temp_df.drop(drop_cols, axis=1)
    access_df = pd.concat([access_df, temp_df], ignore_index=True, sort=False)


Processing:  API_EG.ELC.ACCS.RU.ZS_DS2_Access to Electricity Rural.xls
Processing:  API_EG.ELC.ACCS.UR.ZS_DS2_Access to Electricity Urban.xls
Processing:  API_EG.ELC.ACCS.ZS_DS2_Access to Electricity.xls


In [21]:
# Display statistics for electricity access dataframe
print('Access DataFrame Shape: ', access_df.shape)
print('Unique Indicators: ', access_df['Indicator Name'].nunique())

Access DataFrame Shape:  (792, 63)
Unique Indicators:  3


### Clean up indicator names

In [22]:
# Apply abbreviated indicator names
for i in range(len(access_df)):
    access_df.iloc[i,2] = list(indicators.loc[access_df.iloc[i,2] == indicators['Indicator Name']]['Abbreviated Name'])[0]
    

In [23]:
# Drop columns that are too old for scoring system
drops = ['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970',
       '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979',
       '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009']

access_df = access_df.drop(labels=drops, axis=1)

### Append country codes

In [24]:
# Identify corrections to the access_df alpha-3 country values
a_spec = {'Kosovo':'RKS', 
          'World':'GLOBAL',
         'West Bank and Gaza':'PSE'}

In [25]:
# Apply alpha3 corrections to access_df
for idx, val in enumerate(access_df['Country Name']):
    if val in list(a_spec.keys()):
        access_df.iloc[idx,1] = a_spec[val]

### Evaluate overlap between Electrictiy Access and BMI datasets

In [26]:
# Provide overlap summary
bmi_notin_access = list(set(bmi_df['Country Code']) - set(access_df['Country Code']))
print('Values in BMI dataframe but not in Access dataframe: ', len(bmi_notin_access))
access_notin_bmi = list(set(access_df['Country Code']) - set(bmi_df['Country Code']))
print('Values in Access dataframe but not in BMI dataframe: ', len(access_notin_bmi))

Values in BMI dataframe but not in Access dataframe:  4
Values in Access dataframe but not in BMI dataframe:  65


In [27]:
# Remove non-overlapping countries
access_df.drop(access_df[access_df['Country Code'].isin(access_notin_bmi)].index, inplace=True)
bmi_df.drop(bmi_df[bmi_df['Country Code'].isin(bmi_notin_access)].index, inplace=True)

print('Non-overlapping countries remaining:')
print(list(set(access_df['Country Code']) - set(bmi_df['Country Code'])))
print(list(set(bmi_df['Country Code']) - set(access_df['Country Code'])))

Non-overlapping countries remaining:
[]
[]


### Select 2016-2018 data

Due to several missing values in 2019 column, only go up to 2018.

In [28]:
access_df = access_df[['Country Name', 'Country Code', 'Indicator Name', '2015', '2016', '2017','2018', '2019']]
bmi_df = bmi_df[['Country Name', 'Country Code', 'Indicator Name', '2015', '2016', '2017','2018', '2019']]

In [29]:
# Verify correct format of electricity access dataframe
access_df.head()

,Country Name,Country Code,Indicator Name,2015,2016,2017,2018,2019
0,Aruba,ABW,ACCESS_RURAL_PERC,100.000000,100.00000,100.000000,100.000000,NaN
1,Afghanistan,AFG,ACCESS_RURAL_PERC,64.573354,97.09936,97.091973,98.272872,NaN
2,Angola,AGO,ACCESS_RURAL_PERC,3.815070,NaN,NaN,NaN,NaN
3,Albania,ALB,ACCESS_RURAL_PERC,100.000000,100.00000,100.000000,100.000000,NaN
6,United Arab Emirates,ARE,ACCESS_RURAL_PERC,100.000000,100.00000,100.000000,100.000000,NaN


In [30]:
# Verify correct format of BMI dataframe
bmi_df.head()

,Country Name,Country Code,Indicator Name,2015,2016,2017,2018,2019
48,Bahrain,BHR,GEN_TOTAL_TWH,26.1,27.3,28.5,29.7,39.8
49,Bahrain,BHR,GEN_TOTAL_YOY,6.293,4.483,4.684,4.186,33.887
50,Bahrain,BHR,GEN_TOTAL_KWHPP,19025.4,19126.3,19107.1,18951,24264.1
51,Bahrain,BHR,GEN_THERMAL_TWH,26.094,27.264,28.541,29.736,39.815
52,Bahrain,BHR,GEN_THERMAL_YOY,6.294,4.484,4.684,4.186,33.894


# Process RISE Files

In [31]:
# Read in RISE data
f = rise_file
path = './Raw Data/' + f
rise_df = pd.read_excel(path, sheet_name='Overall Scores')
sheets = ['Electricity Access', 'Energy Efficiency', 'Renewable Energy']

for s in sheets:
    temp_df = pd.read_excel(path, sheet_name=s) #, usecols=use_cols)
    rise_df = rise_df.merge(temp_df, left_on='Countries', right_on='Countries' )

### Clean up RISE indicator names

In [32]:
# Apply abbreviated indicator names to RISE dataframe
for val_col in rise_df.columns:
    rise_df = rise_df.rename(columns={val_col: indicators.loc[indicators['Indicator Name'] == val_col, 'Abbreviated Name'].iloc[0]})
    

In [33]:
# Verify indicator names changed correctly
rise_df.head()

,Country Name,RISE_ACCESS,RISE_EFFICIENCY,RISE_RENEWABLES,RISE_OVERALL,RISE_EA1,RISE_EA2,RISE_EA3,RISE_EA4,RISE_EA5,...,RISE_EE11,RISE_EE12,RISE_EE13,RISE_RE1,RISE_RE2,RISE_RE3,RISE_RE4,RISE_RE5,RISE_RE6,RISE_RE7
0,East Asia & Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cambodia,80.25,44.92,41.29,55.49,86.0,33.0,100.0,78.0,100.0,...,0.0,0.0,50.0,100.0,15.0,18.0,0.0,30.0,76.0,50.0
2,China,100.00,71.00,66.43,79.14,0.0,0.0,0.0,0.0,0.0,...,57.0,33.0,50.0,100.0,71.0,71.0,17.0,79.0,77.0,50.0
3,Indonesia,67.50,26.15,53.71,49.12,67.0,42.0,67.0,47.0,67.0,...,0.0,0.0,0.0,100.0,66.0,31.0,33.0,13.0,83.0,50.0
4,Lao PDR,45.50,11.38,30.57,29.15,42.0,33.0,50.0,47.0,44.0,...,0.0,17.0,0.0,100.0,55.0,27.0,8.0,7.0,17.0,0.0


In [34]:
# Drop region labels
rise_df.dropna(subset=['RISE_OVERALL'], inplace=True)

### Append country codes

In [35]:
# Create alpha-3 country code column
rise_df['Country Code'] = ''

In [36]:
missed = [] # list of countries that didn't have a matching country code
# dictionary of incorrect country names/codes
catches = {'Lao PDR':'LAO',
 'Venezuela, RB':'VEN',
 'Egypt, Arab Rep.':'EGY',
 'Iran, Islamic Rep.':'IRN',
 'Yemen, Rep.':'YEM',
 'Korea, Rep.':'KOR',
 'Congo, Dem. Rep.': 'COD',
 'Congo, Rep.':'COG',
 'Niger': 'NER'}

print('Alpha-3 country encoding progress:')
print('|', '-'*int(math.ceil((len(rise_df) / 10))), '|')
print('  ', end='')

for idx, val in enumerate(rise_df['Country Name']):
    if idx % 10 == 0:
        print('*',end='')
    try:
        alpha3 = pycountry.countries.search_fuzzy(val)[0].alpha_3
        rise_df.iloc[idx,-1] = alpha3
    except:
        try:
            alpha3 = catches[val]
            rise_df.iloc[idx,-1] = alpha3
        except:
            missed.append(val)     

# one-off correction
rise_df.loc[rise_df['Country Name'] == 'Niger','Country Code'] = 'NER'

# report if missing any countries
print('\n Missed countries: ', missed)

Alpha-3 country encoding progress:
| -------------- |
  **************
 Missed countries:  []


In [37]:
# Verify country code column
rise_df.head()

,Country Name,RISE_ACCESS,RISE_EFFICIENCY,RISE_RENEWABLES,RISE_OVERALL,RISE_EA1,RISE_EA2,RISE_EA3,RISE_EA4,RISE_EA5,...,RISE_EE12,RISE_EE13,RISE_RE1,RISE_RE2,RISE_RE3,RISE_RE4,RISE_RE5,RISE_RE6,RISE_RE7,Country Code
1,Cambodia,80.25,44.92,41.29,55.49,86.0,33.0,100.0,78.0,100.0,...,0.0,50.0,100.0,15.0,18.0,0.0,30.0,76.0,50.0,KHM
2,China,100.00,71.00,66.43,79.14,0.0,0.0,0.0,0.0,0.0,...,33.0,50.0,100.0,71.0,71.0,17.0,79.0,77.0,50.0,CHN
3,Indonesia,67.50,26.15,53.71,49.12,67.0,42.0,67.0,47.0,67.0,...,0.0,0.0,100.0,66.0,31.0,33.0,13.0,83.0,50.0,IDN
4,Lao PDR,45.50,11.38,30.57,29.15,42.0,33.0,50.0,47.0,44.0,...,17.0,0.0,100.0,55.0,27.0,8.0,7.0,17.0,0.0,LAO
5,Malaysia,100.00,56.46,57.71,71.39,0.0,0.0,0.0,0.0,0.0,...,50.0,0.0,100.0,71.0,70.0,58.0,32.0,73.0,0.0,MYS


### Melt rise data for consistency with previous data formats

In [38]:
# Store all electricity access data in 2017 column
rise_df = pd.melt(rise_df, id_vars=['Country Name', 'Country Code'], var_name='Indicator Name', value_name='2017')
rise_df.sort_values(by=['Country Code'], inplace=True)

In [39]:
# Verify RISE dataframe melted correctly
rise_df.head()

,Country Name,Country Code,Indicator Name,2017
3522,Afghanistan,AFG,RISE_RE2,12.00
486,Afghanistan,AFG,RISE_OVERALL,24.47
1542,Afghanistan,AFG,RISE_EA8,77.00
3654,Afghanistan,AFG,RISE_RE3,13.00
2598,Afghanistan,AFG,RISE_EE8,0.00


# Process Survey File

In [40]:
# Read in survey data
f = survey_file
path = './Raw Data/' + f
survey_df = pd.read_excel(path, sheet_name='Sheet1')
survey_df_lookup = pd.read_excel(path, sheet_name='Sheet2')
survey_df.drop(list(range(8)), inplace=True) # first 8 rows are not valid countries

### Append country codes

In [41]:
survey_df.columns = list(survey_df_lookup['New Name']) # fix survey column naming

# Create alpha-3 country codes
survey_df['Country Code'] = ''
col_names = survey_df.columns
survey_df = survey_df[[col_names[0]] + [col_names[-1]] + list(col_names[1:-1])]

In [42]:
missed = [] # list of countries that don't have a valid alpha-3 code
# dictionary of countries that don't have a valid country name
catches = {
    'Congo, Dem. Rep.':'COD', 
    'Congo, Rep.':'COG', 
    'Lao PDR':'LAO', 
    'St. Kitts and Nevis':'KNA', 
    'St. Lucia':'LCA', 
    'St. Vincent and the Grenadines':'VCT', 
    'West Bank and Gaza':'PSE'
}

print('Alpha-3 country encoding progress:')
print('|', '-'*int(math.ceil((len(survey_df) / 10))), '|')
print('  ', end='')

for idx, val in enumerate(survey_df['Country Name']):
    if idx % 10 == 0:
        print('*',end='')
    try:
        alpha3 = pycountry.countries.search_fuzzy(val)[0].alpha_3
        survey_df.iloc[idx,1] = alpha3
    except:
        try:
            alpha3 = catches[val]
            survey_df.iloc[idx,1] = alpha3
        except:
            missed.append(val)     

# One-off corrections:
survey_df.loc[survey_df['Country Name'] == 'Niger','Country Code'] = 'NER'
survey_df.loc[survey_df['Country Name'] == 'Nigeria','Country Code'] = 'NGA'
survey_df.loc[survey_df['Country Name'] == 'Kosovo','Country Code'] = 'RKS'

print('\n Missed countries: ', missed)

Alpha-3 country encoding progress:
| --------------- |
  ***************
 Missed countries:  []


In [43]:
# Verify country codes applied correctly
survey_df.head()

,Country Name,Country Code,SURV_OUTAGE_PERC,SURV_OUTAGE_QTY,SURV_OUTAGE_DUR,SURV_OUTAGE_LOSS_PERC,SURV_GEN,SURV_GEN_USE,SURV_ELEC_CONNECT,SURV_ELEC_CONSTRAINT,SURV_WATER_PERC,SURV_WATER_QTY,SURV_SHIP,SURV_TRANS
8,Afghanistan,AFG,70.4,11.5,3.8,9.6,48.0,38.3,111.3,65.8,8.9,0.9,1.7,42.1
9,Albania,ALB,58.7,1.5,1.3,1.9,49.4,11.9,18.6,43.0,10.8,0.1,1.4,10.7
10,Angola,AGO,87.7,4.7,13.5,12.6,79.0,27.3,7.7,35.7,46.1,2.4,3.3,25.3
11,Antigua and Barbuda,ATG,95.5,2.8,2.2,0.2,22.4,3.3,6.7,45.1,3.0,0.0,NaN,24.4
12,Argentina,ARG,65.1,0.8,5.2,0.8,17.8,3.5,53.5,47.2,10.4,0.2,0.6,22.8


### Melt survey data for consistency with previous data formats

In [44]:
# Store all electricity access data in 2017 column
survey_df = pd.melt(survey_df, id_vars=['Country Name', 'Country Code'], var_name='Indicator Name', value_name='2017')
survey_df.sort_values(by=['Country Code'], inplace=True)

In [45]:
# verify survey dataframe melted correctly
survey_df.head()

,Country Name,Country Code,Indicator Name,2017
0,Afghanistan,AFG,SURV_OUTAGE_PERC,70.4
720,Afghanistan,AFG,SURV_GEN_USE,38.3
432,Afghanistan,AFG,SURV_OUTAGE_LOSS_PERC,9.6
864,Afghanistan,AFG,SURV_ELEC_CONNECT,111.3
1008,Afghanistan,AFG,SURV_ELEC_CONSTRAINT,65.8


# Process CO2 File

In [46]:
# Read in data
path = './Raw Data/' + co2_file
co2_df = pd.read_excel(path, sheet_name='Data', header=3, usecols=['Country Code', '2014'])
co2_df = co2_df.rename(columns={'2014':'CO2_EMISS_TONSPP'})

# Melt data into 2017 column
co2_df = pd.melt(co2_df, id_vars='Country Code', var_name='Indicator Name', value_name='2017')

In [47]:
# Verify CO2 dataframe in correct format
co2_df

,Country Code,Indicator Name,2017
0,ABW,CO2_EMISS_TONSPP,8.410064
1,AFG,CO2_EMISS_TONSPP,0.293946
2,AGO,CO2_EMISS_TONSPP,1.290307
3,ALB,CO2_EMISS_TONSPP,1.978763
4,AND,CO2_EMISS_TONSPP,5.832906
...,...,...,...
259,XKX,CO2_EMISS_TONSPP,NaN
260,YEM,CO2_EMISS_TONSPP,0.878996
261,ZAF,CO2_EMISS_TONSPP,8.979062
262,ZMB,CO2_EMISS_TONSPP,0.292412


# Process Market Potential File

In [48]:
# Read in data
path = './Raw Data/' + mktpot_file
mktpot_df = pd.read_excel(path, sheet_name='Market Potential')

# Melt data into 2017 column
mktpot_df = pd.melt(mktpot_df, id_vars='Country Code', var_name='Indicator Name', value_name='2017')

# Create the Consolidated Dataframe

1. Check for duplicate countries
2. Check datatypes for each column
3. Check for NaNs in each df
4. Create consolidated dataframe


### Check for duplicate countries

In [49]:
print('BMI No Duplicates: ', bmi_df['Country Code'].nunique() == bmi_df['Country Name'].nunique())
print('Electricity Access No Duplicates: ', access_df['Country Code'].nunique() == access_df['Country Name'].nunique())
print('RISE No Duplicates: ', rise_df['Country Name'].nunique() == rise_df['Country Code'].nunique())
print('Survey Data No Duplicates: ', survey_df['Country Code'].nunique() == survey_df['Country Name'].nunique())
print('Market Potential No Duplicates: ', mktpot_df['Country Code'].nunique() == len(mktpot_df))
print('CO2 Data No Duplicates: ', co2_df['Country Code'].nunique() == len(co2_df))

BMI No Duplicates:  True
Electricity Access No Duplicates:  True
RISE No Duplicates:  True
Survey Data No Duplicates:  True
Market Potential No Duplicates:  True
CO2 Data No Duplicates:  True


### Create consolidated df

In [50]:
# Label data sources as a new column in each dataset
bmi_df['Source'] = 'BMI'
access_df['Source'] = 'ACCESS'
rise_df['Source'] = 'RISE'
survey_df['Source'] = 'SURVEY'
mktpot_df['Source'] = 'ICON'
co2_df['Source'] = 'BMI'

In [51]:
df = pd.concat([bmi_df, access_df, rise_df, survey_df, mktpot_df, co2_df], sort=True)

# rearrange df
df = df[['Country Name', 'Country Code', 'Indicator Name', 'Source','2015', '2016', '2017','2018', '2019']]

# sort alphabetically by country code
df.sort_values(by=['Country Code'], inplace=True)

# fix the indexes
df.reset_index(drop=True, inplace=True)

In [52]:
# verify consolidated dataframe created
df

,Country Name,Country Code,Indicator Name,Source,2015,2016,2017,2018,2019
0,Aruba,ABW,ACCESS_URBAN_PERC,ACCESS,100,100,100,100,NaN
1,Aruba,ABW,RI_LOGISTIC,BMI,46.3,45.5,49.4,46.8,48.8
2,Aruba,ABW,RI_TRADE,BMI,65,62.5,61.8,60.1,61.5
3,Aruba,ABW,ACCESS_PERC,ACCESS,100,100,100,100,NaN
4,Aruba,ABW,POP_RURAL_PERC,BMI,56.9,56.8,56.7,56.5,56.4
...,...,...,...,...,...,...,...,...,...
15059,Zimbabwe,ZWE,RISE_EE11,RISE,NaN,NaN,0,NaN,NaN
15060,Zimbabwe,ZWE,RISE_EE10,RISE,NaN,NaN,0,NaN,NaN
15061,Zimbabwe,ZWE,RISE_EE9,RISE,NaN,NaN,0,NaN,NaN
15062,Zimbabwe,ZWE,RISE_RE2,RISE,NaN,NaN,57,NaN,NaN


### Change column datatypes to float

In [53]:
for c in ['2015', '2016', '2017','2018', '2019']:
    df[c] = df[c].replace('-',np.nan)
    df[c] = df[c].astype(float)

In [54]:
# verify datatypes are float for each year group
df.dtypes

Country Name       object
Country Code       object
Indicator Name     object
Source             object
2015              float64
2016              float64
2017              float64
2018              float64
2019              float64
dtype: object

### Check for NaNs

(do nothing with them for now)

In [55]:
# Display percentage of missing values in each column
df[(df['Source'] == 'BMI') | (df['Source'] == 'ACCESS')].isna().mean()

Country Name      0.029623
Country Code      0.000000
Indicator Name    0.000000
Source            0.000000
2015              0.033214
2016              0.034111
2017              0.008528
2018              0.039722
2019              0.109066
dtype: float64

In [56]:
# display final shape of dataframe
df.shape

(15064, 9)

### Pivot into final dataframe format

Rows should be country codes and columns should be indicator names

In [57]:
df_piv = df.pivot(index='Country Code', columns='Indicator Name', values='2017')

In [58]:
# Verify final dataframe shape
df_piv

Indicator Name,ACCESS_PERC,ACCESS_RURAL_PERC,ACCESS_URBAN_PERC,CAPACITY_HYDRO_MW,CAPACITY_HYDRO_PERC,CAPACITY_HYDRO_YOY,CAPACITY_MW,CAPACITY_NUC_MW,CAPACITY_NUC_PERC,CAPACITY_NUC_YOY,...,SURV_GEN,SURV_GEN_USE,SURV_OUTAGE_DUR,SURV_OUTAGE_LOSS_PERC,SURV_OUTAGE_PERC,SURV_OUTAGE_QTY,SURV_SHIP,SURV_TRANS,SURV_WATER_PERC,SURV_WATER_QTY
Country Code,,,,,,,,,,,,,,,,,,,,,
ABW,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,97.700000,97.091973,99.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.0,38.3,3.8,9.6,70.4,11.5,1.7,42.1,8.9,0.9
AGO,41.962894,NaN,71.886459,2420.0,64.3,90.6,3764.4,0.0,0.0,NaN,...,79.0,27.3,13.5,12.6,87.7,4.7,3.3,25.3,46.1,2.4
ALB,100.000000,100.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.4,11.9,1.3,1.9,58.7,1.5,1.4,10.7,10.8,0.1
AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YEM,79.200000,68.766799,97.735039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80.5,48.1,4.5,19.7,93.8,38.8,2.4,33.9,54.0,11.6
ZAF,84.400000,70.314530,91.704765,3595.0,6.8,5.6,52843.1,1860.0,3.5,0.0,...,18.4,10.9,4.5,1.6,44.9,0.9,1.4,3.9,5.4,0.1


In [59]:
print('Number of rows with >25% missing data: ', (df_piv.isna().sum(axis=1)/df_piv.shape[1] > 0.25).sum())

Number of rows with >25% missing data:  166


# Export all data to excel spreadsheet

In [60]:
# create a dataframe to show the association of indicator names and their abbreviated form
indicators_clean = pd.DataFrame()
indicators_clean['Abbreviated Name'] = list(df_piv.columns)
indicators_clean['Indicator Name'] = ''

for ind_idx, ind_val in enumerate(indicators_clean['Abbreviated Name']):
    indicators_clean.iloc[ind_idx, 1] = indicators.loc[indicators['Abbreviated Name'] == ind_val , 'Indicator Name'].iloc[0]
    

In [61]:
# Verify indicators have abbreviated and full form in dataframe
indicators_clean

,Abbreviated Name,Indicator Name
0,ACCESS_PERC,Access to electricity (% of population)
1,ACCESS_RURAL_PERC,"Access to electricity, rural (% of rural popul..."
2,ACCESS_URBAN_PERC,"Access to electricity, urban (% of urban popul..."
3,CAPACITY_HYDRO_MW,"ELECTRICITY CAPACITY: Capacity, Hydropower, MW"
4,CAPACITY_HYDRO_PERC,"ELECTRICITY CAPACITY: Capacity, Hydropower, % ..."
...,...,...
107,SURV_OUTAGE_QTY,Number of electrical outages in a typical month
108,SURV_SHIP,Proportion of products lost to breakage or spo...
109,SURV_TRANS,Percent of firms identifying transportation as...
110,SURV_WATER_PERC,Percent of firms experiencing water insufficie...


In [62]:
# Save all to excel
with pd.ExcelWriter('ALL DATA.xlsx') as writer:  
    df_piv.to_excel(writer, sheet_name='2017 Snapshot')
    df.to_excel(writer, sheet_name='ALL DATA')
    indicators_clean.to_excel(writer, sheet_name='Indicator Definitions')


# DELETE ALL BELOW